In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
from tqdm.notebook import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [ ]:
sub=pd.read_csv("/content/submission_sample.csv", sep=";", engine="python")

In [ ]:
train_df=pd.read_csv("/content/Xy_train.csv", sep=";",engine="python")
test_df=pd.read_csv("/content/X_test.csv", sep=";",engine="python")


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
Id=test_df["_id"]

In [ ]:
train_df.shape

In [ ]:
train_df["y"].nunique()

In [ ]:
train_df.isna().sum().sort_values(ascending =False).head(20)

In [ ]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

# Features engineering

*I choose to drop text_rows because after encoding them and use other method like tfidf and bag of words , it didn't give me better result*

In [ ]:
train_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)
test_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)


In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
import datetime as dt
train_df["date_0"]=pd.to_datetime(train_df["date_0"])
train_df['Date_year'] =train_df["date_0"].dt.year
train_df['Date_month'] = train_df["date_0"].dt.month
train_df['Date_day'] = train_df["date_0"].dt.day
train_df.drop("date_0", axis=1, inplace=True)

In [ ]:
test_df["date_0"]=pd.to_datetime(test_df["date_0"])
test_df['Date_year'] =test_df["date_0"].dt.year
test_df['Date_month'] = test_df["date_0"].dt.month
test_df['Date_day'] = test_df["date_0"].dt.day
test_df.drop("date_0", axis=1, inplace=True)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

# Mice

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)
imp_train_df=mice.fit_transform(train_df)
imp_train_df=pd.DataFrame(imp_train_df,columns=train_df.columns)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
imp_test_df=mice.fit_transform(test_df)
imp_test_df=pd.DataFrame(imp_test_df,columns=test_df.columns)

In [ ]:
imp_train_df.isna().sum()

In [ ]:
imp_train_df["mean"]=imp_train_df.mean(axis=1)
imp_train_df["count"]=imp_train_df.count(axis=1)
imp_train_df["std"]=imp_train_df.std(axis=1)
imp_train_df["min"]=imp_train_df.min(axis=1)
imp_train_df["max"]=imp_train_df.max(axis=1)
imp_train_df["kurt"]=imp_train_df.kurt(axis=1)
imp_train_df["skew"]=imp_train_df.skew(axis=1)
imp_train_df["q1"]=imp_train_df.quantile(q=0.1,axis=1)
imp_train_df["q2"]=imp_train_df.quantile(q=0.2,axis=1)
imp_train_df["q25"]=imp_train_df.quantile(q=0.25,axis=1)
imp_train_df["q3"]=imp_train_df.quantile(q=0.3,axis=1)
imp_train_df["q4"]=imp_train_df.quantile(q=0.4,axis=1)
imp_train_df["q5"]=imp_train_df.quantile(q=0.5,axis=1)
imp_train_df["q6"]=imp_train_df.quantile(q=0.6,axis=1)
imp_train_df["q7"]=imp_train_df.quantile(q=0.7,axis=1)
imp_train_df["q75"]=imp_train_df.quantile(q=0.75,axis=1)
imp_train_df["q8"]=imp_train_df.quantile(q=0.8,axis=1)
imp_train_df["q9"]=imp_train_df.quantile(q=0.9,axis=1)

In [ ]:
imp_test_df["mean"]=imp_test_df.mean(axis=1)
imp_test_df["count"]=imp_test_df.count(axis=1)
imp_test_df["std"]=imp_test_df.std(axis=1)
imp_test_df["min"]=imp_test_df.min(axis=1)
imp_test_df["max"]=imp_test_df.max(axis=1)
imp_test_df["kurt"]=imp_test_df.kurt(axis=1)
imp_test_df["skew"]=imp_test_df.skew(axis=1)
imp_test_df["q1"]=imp_test_df.quantile(q=0.1,axis=1)
imp_test_df["q2"]=imp_test_df.quantile(q=0.2,axis=1)
imp_test_df["q25"]=imp_test_df.quantile(q=0.25,axis=1)
imp_test_df["q3"]=imp_test_df.quantile(q=0.3,axis=1)
imp_test_df["q4"]=imp_test_df.quantile(q=0.4,axis=1)
imp_test_df["q5"]=imp_test_df.quantile(q=0.5,axis=1)
imp_test_df["q6"]=imp_test_df.quantile(q=0.6,axis=1)
imp_test_df["q7"]=imp_test_df.quantile(q=0.7,axis=1)
imp_test_df["q75"]=imp_test_df.quantile(q=0.75,axis=1)
imp_test_df["q8"]=imp_test_df.quantile(q=0.8,axis=1)
imp_test_df["q9"]=imp_test_df.quantile(q=0.9,axis=1)

In [ ]:
y_train=imp_train_df["y"]

In [ ]:
imp_train_df=imp_train_df.drop("y",axis=1)

In [ ]:
imp_train_df.shape, imp_test_df.shape

# Scaling 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(imp_train_df)
X_test = mms.transform(imp_test_df)


In [ ]:
mms.fit_transform(y_train)

In [ ]:
X_test.shape, X_train.shape

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.1)

****

# Modeling

**RandomForestRegressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(min_samples_split=15)
Rf.fit(xtrain, ytrain)
ypred=Rf.predict(xtest)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, ypred)

In [ ]:
mean_squared_error(ytest, ypred, squared=True)

In [ ]:
Rf.fit(xtest, ytest)
ypred=Rf.predict(xtest)

In [ ]:
mean_squared_error(ytest, ypred, squared=True)

In [ ]:
# y_pred1 = Rf.predict(x_test) 
# y_pred_original1 = mms.inverse_transform(y_pred1.reshape(-1,1))
# y_pred_original1=y_pred_original1.tolist()
# y_pred_original1=[i[0] for i in y_pred_original1]
# submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original1})
# submission.to_csv('submission0.csv', index=False)

****

**XGBRegressor : the best result**

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror')
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model,xtrain,ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MSE: %.8f (%53f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model.fit(xtrain, ytrain)
yhat = model.predict(xtest)

In [ ]:
mean_squared_error(ytest, yhat, squared=True)

In [ ]:
model.fit(X_train, y_train)
y_pred4 = model.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))


In [ ]:
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original4})
submission.to_csv('submission1.csv', index=False)

****

**LGBMRegressor**

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model1 = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model1, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.5f (%.5f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model1 = LGBMRegressor()
model1.fit(xtrain, ytrain)
yhatt = model1.predict(xtest)
mean_squared_error(ytest, yhatt)

In [ ]:
# model1.fit(X_train, y_train)
# y_pred2 = model1.predict(X_test) 
# y_pred_original2 = mms.inverse_transform(y_pred2.reshape(-1,1))
# y_pred_original2=y_pred_original4.tolist()
# y_pred_original2=[i[0] for i in y_pred_original2]
# submission = pd.DataFrame({'_id	': Id,'y_pred': y_pred_original2})
# submission.to_csv('submission13.csv', index=False)

**GradientBoostingRegressor**

In [ ]:
from sklearn import  ensemble
params = {
    "n_estimators": 500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(xtrain, ytrain)
mse = mean_squared_error(ytest, reg.predict(xtest))
mse

In [ ]:
# y_pred3 = reg.predict(X_test) 
# y_pred_original3 = mms.inverse_transform(y_pred3.reshape(-1,1))
# y_pred_original3=y_pred_original3.tolist()
# y_pred_original3=[i[0] for i in y_pred_original3]
# submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original3})
# submission.to_csv('submission14.csv', index=False)